In [1]:
from os import path
import mlrun

# Set the base project name
project_name_base = 'test-mlrun'

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name_base, context="./", user_project=True)

# Display the current project name
project_name = project.metadata.name
print(f'Full project name: {project_name}')

> 2022-03-13 08:39:20,459 [info] loaded project test-mlrun from MLRun DB
Full project name: test-mlrun-jovyan


In [2]:
# mlrun: start-code

In [3]:
from cloudpickle import load
import numpy as np
from typing import List
import mlrun

class ClassifierModel(mlrun.serving.V2ModelServer):
    def load(self):
        """load and initialize the model and/or other elements"""
        model_file, extra_data = self.get_model('.pkl')
        self.model = load(open(model_file, 'rb'))

    def predict(self, body: dict) -> List:
        """Generate model predictions from sample."""
        feats = np.asarray(body['inputs'])
        result: np.ndarray = self.model.predict(feats)
        return result.tolist()

In [4]:
# mlrun: end-code

In [5]:
serving_fn = mlrun.code_to_function('serving', kind='serving',image='mlrun/mlrun')
serving_fn.spec.default_class = 'ClassifierModel'

In [11]:
model_file = project.get_artifact_uri('train-iris-train_iris_Model') 
serving_fn.add_model('my_model',model_path=model_file)

In [12]:
my_data = '''{"inputs":[[5.1, 3.5, 1.4, 0.2],[7.7, 3.8, 6.7, 2.2]]}'''

In [13]:
server = serving_fn.to_mock_server()
server.test("/v2/models/my_model/infer", body=my_data)

> 2022-03-13 08:50:27,251 [info] model my_model was loaded
> 2022-03-13 08:50:27,251 [info] Loaded ['my_model']


X does not have valid feature names, but RandomForestClassifier was fitted with feature names


{'id': 'a6cdc1263054490a8a081e3eafdc349e',
 'model_name': 'my_model',
 'outputs': [0, 2]}

In [14]:
function_address = serving_fn.deploy()

> 2022-03-13 08:50:29,453 [info] Starting remote function deploy
2022-03-13 08:50:29  (info) Deploying function
2022-03-13 08:50:29  (info) Building
2022-03-13 08:50:29  (info) Staging files and preparing base images
2022-03-13 08:50:29  (warn) Python 3.6 runtime is deprecated and will soon not be supported. Please migrate your code and use Python 3.7 runtime (`python:3.7`) or higher
2022-03-13 08:50:29  (info) Building processor image
2022-03-13 08:51:29  (info) Build complete
2022-03-13 08:51:37  (info) Function deploy complete
> 2022-03-13 08:51:38,077 [info] successfully deployed function: {'internal_invocation_urls': ['nuclio-test-mlrun-jovyan-serving.mlrun.svc.cluster.local:8080'], 'external_invocation_urls': ['localhost:30240']}
